# **PDF Chatbot: A RAG Pipeline**

This notebook demonstrates a simple Retrieval Augmented Generation (RAG) pipeline built within Google Colab, allowing you to interactively chat with your PDF documents. The goal is to process PDF files, create a searchable knowledge base from their content, and use a Large Language Model (LLM) to answer questions based on the retrieved information.

The pipeline consists of the following main steps:

1.  **PDF Processing:** Extracting text content from PDF files using `PyMuPDF`.
2.  **Text Chunking:** Splitting the extracted text into smaller, manageable chunks.
3.  **Embeddings:** Converting text chunks into numerical vector representations using a Sentence Transformer model.
4.  **Vector Store:** Creating a FAISS index for efficient storage and retrieval of text embeddings.
5.  **Retrieval:** Searching the vector store to find the most relevant text chunks for a given query.
6.  **LLM Setup:** Loading and configuring a Large Language Model (LLM) for text generation (using `transformers`), including optional quantization for memory efficiency.
7.  **Generation:** Using the LLM to generate an answer based on the user's query and the retrieved relevant text chunks.

Key libraries used in this pipeline include:

*   `PyMuPDF` (fitz) for PDF text extraction.
*   `sentence-transformers` for creating text embeddings.
*   `transformers` and `accelerate`/`bitsandbytes` for loading and managing the LLM.
*   `faiss-cpu` for building the vector index and performing similarity search.
*   `pandas` and `numpy` for data handling.

## 1. Install Required Packages


In [ ]:
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install -q transformers accelerate bitsandbytes
%pip install -q sentence-transformers
%pip install -q PyMuPDF tqdm pandas
%pip install -q faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 72.3 MB/s eta 0:00:00


## 2. Import Libraries


In [ ]:
import torch
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import faiss
from tqdm.auto import tqdm
import re
import warnings
warnings.filterwarnings('ignore')


## 3. PDF Processing Functions


In [ ]:
def text_formatter(text: str) -> str:
    """Clean and format text"""
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace with single space
    text = re.sub(r'\n+', '\n', text)  # Replace multiple newlines with single newline
    return text.strip()

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """Extract text from PDF and return list of pages"""
    doc = fitz.open(pdf_path)
    pages_text = []

    for page_num in tqdm(range(len(doc)), desc="Reading PDF pages"):
        page = doc[page_num]
        text = page.get_text()
        text = text_formatter(text)

        if text:  # Only add non-empty pages
            pages_text.append({
                'page_number': page_num + 1,
                'text': text,
                'char_count': len(text)
            })

    doc.close()
    return pages_text


## 4. Text Chunking


In [ ]:
def split_text_into_chunks(text: str, chunk_size: int = 1000, overlap: int = 200) -> list[str]:
    """Split text into overlapping chunks"""
    if len(text) <= chunk_size:
        return [text]

    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size

        # Try to break at sentence boundary
        if end < len(text):
            # Look for sentence endings within the last 100 characters
            search_start = max(start, end - 100)
            sentence_end = text.rfind('.', search_start, end)
            if sentence_end > start:
                end = sentence_end + 1

        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)

        start = end - overlap
        if start >= len(text):
            break

    return chunks

def create_chunks_from_pages(pages: list[dict], chunk_size: int = 1000, overlap: int = 200) -> list[dict]:
    """Create chunks from PDF pages"""
    all_chunks = []

    for page in tqdm(pages, desc="Creating chunks"):
        chunks = split_text_into_chunks(page['text'], chunk_size, overlap)

        for i, chunk in enumerate(chunks):
            all_chunks.append({
                'page_number': page['page_number'],
                'chunk_id': f"page_{page['page_number']}_chunk_{i+1}",
                'text': chunk,
                'char_count': len(chunk)
            })

    return all_chunks


## 5. Embeddings and Vector Store


In [ ]:
def create_embeddings(chunks: list[dict], model_name: str = "all-MiniLM-L6-v2") -> tuple:
    """Create embeddings for text chunks"""
    print(f"Loading embedding model: {model_name}")
    embedding_model = SentenceTransformer(model_name)

    # Extract texts
    texts = [chunk['text'] for chunk in chunks]

    print("Creating embeddings...")
    embeddings = embedding_model.encode(texts, show_progress_bar=True)

    return embeddings, embedding_model

def create_faiss_index(embeddings: np.ndarray) -> faiss.Index:
    """Create FAISS index for similarity search"""
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Inner product (cosine similarity)

    # Normalize embeddings for cosine similarity
    faiss.normalize_L2(embeddings)
    index.add(embeddings.astype('float32'))

    return index


## 6. Retrieval Function


In [ ]:
def retrieve_relevant_chunks(query: str,
                           embedding_model: SentenceTransformer,
                           index: faiss.Index,
                           chunks: list[dict],
                           top_k: int = 5) -> list[dict]:
    """Retrieve most relevant chunks for a query"""
    # Create query embedding
    query_embedding = embedding_model.encode([query])
    faiss.normalize_L2(query_embedding)

    # Search for similar chunks
    scores, indices = index.search(query_embedding.astype('float32'), top_k)

    # Get relevant chunks with scores
    relevant_chunks = []
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        chunk = chunks[idx].copy()
        chunk['similarity_score'] = float(score)
        chunk['rank'] = i + 1
        relevant_chunks.append(chunk)

    return relevant_chunks


## 7. LLM Setup


In [ ]:
def setup_llm(model_name: str = "microsoft/DialoGPT-medium", use_quantization: bool = True):
    """Setup LLM for text generation"""
    print(f"Loading LLM: {model_name}")

    # Setup quantization for memory efficiency
    if use_quantization and torch.cuda.is_available():
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
    else:
        quantization_config = None

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )

    # Add padding token if not present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer


## 8. RAG Pipeline


In [ ]:
def generate_answer(query: str,
                   relevant_chunks: list[dict],
                   model,
                   tokenizer,
                   max_length: int = 512) -> str:
    """Generate answer using retrieved context"""
    # Create context from relevant chunks
    context = "\n\n".join([chunk['text'] for chunk in relevant_chunks])

    # Create prompt
    prompt = f"""Context: {context}

Question: {query}

Answer: Based on the provided context, """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)

    # Move to same device as model
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer part
    answer = response.split("Answer: Based on the provided context, ")[-1]

    return answer.strip()

def rag_pipeline(query: str,
                embedding_model: SentenceTransformer,
                index: faiss.Index,
                chunks: list[dict],
                model,
                tokenizer,
                top_k: int = 5) -> dict:
    """Complete RAG pipeline"""
    # Retrieve relevant chunks
    relevant_chunks = retrieve_relevant_chunks(query, embedding_model, index, chunks, top_k)

    # Generate answer
    answer = generate_answer(query, relevant_chunks, model, tokenizer)

    return {
        'query': query,
        'answer': answer,
        'relevant_chunks': relevant_chunks,
        'sources': [f"Page {chunk['page_number']}" for chunk in relevant_chunks]
    }


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  # Assuming only one file is uploaded, set the PDF_PATH
  PDF_PATH = "/content/" + fn
  print(f"\nPDF_PATH set to: {PDF_PATH}")

Saving sample_pdf.pdf to sample_pdf.pdf
User uploaded file "sample_pdf.pdf" with length 169506 bytes

PDF_PATH set to: /content/sample_pdf.pdf


## 9. Complete RAG Setup


In [ ]:
# This cell will now use the PDF_PATH variable set by the upload cell

# Step 1: Process PDF
print("Step 1: Processing PDF...")
if 'PDF_PATH' not in globals():
    print("Please upload a PDF file first using the cell above.")
else:
    pages = open_and_read_pdf(PDF_PATH)
    print(f"Extracted {len(pages)} pages")

    # Step 2: Create chunks
    print("\nStep 2: Creating text chunks...")
    chunks = create_chunks_from_pages(pages, chunk_size=1000, overlap=200)
    print(f"Created {len(chunks)} chunks")

    # Step 3: Create embeddings
    print("\nStep 3: Creating embeddings...")
    embeddings, embedding_model = create_embeddings(chunks)
    print(f"Created embeddings with shape: {embeddings.shape}")

    # Step 4: Create FAISS index
    print("\nStep 4: Creating FAISS index...")
    index = create_faiss_index(embeddings)
    print(f"FAISS index created with {index.ntotal} vectors")

    # Step 5: Setup LLM
    print("\nStep 5: Setting up LLM...")
    # Use the TinyLlama model ID
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    model, tokenizer = setup_llm(model_name, use_quantization=True)
    print("LLM setup complete!")

    print("\n✅ RAG pipeline is ready!")

Step 1: Processing PDF...


Reading PDF pages:   0%|          | 0/2 [00:00<?, ?it/s]

Extracted 2 pages

Step 2: Creating text chunks...


Creating chunks:   0%|          | 0/2 [00:00<?, ?it/s]

Created 8 chunks

Step 3: Creating embeddings...
Loading embedding model: all-MiniLM-L6-v2
Creating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Created embeddings with shape: (8, 384)

Step 4: Creating FAISS index...
FAISS index created with 8 vectors

Step 5: Setting up LLM...
Loading LLM: TinyLlama/TinyLlama-1.1B-Chat-v1.0


LLM setup complete!

✅ RAG pipeline is ready!


## 10. Ask Questions


In [ ]:
# Example usage
def ask_question(question: str):
    """Ask a question and get an answer with sources"""
    result = rag_pipeline(question, embedding_model, index, chunks, model, tokenizer)

    print(f"\n❓ Question: {result['query']}")
    print(f"\n💡 Answer: {result['answer']}")
    print(f"\n📚 Sources: {', '.join(result['sources'])}")

    return result

# Ask your questions here
question = "What are the main topics discussed in this document?"
result = ask_question(question)



❓ Question: What are the main topics discussed in this document?

💡 Answer: Context: All You Need" paper) still dominates, new variants and improvements are constantly emerging to make them more efficient and capable of handling longer contexts (e.g., processing entire books at once). • Diffusion Models: This is the architecture that powers most of the current state-of-the-art image generators (like Stable Diffusion and Midjourney). It works by progressively adding noise to data and then learning to reverse the process, creating highly detailed and coherent images from chaos. • Multimodal Architectures: Research is focused on the best ways to fuse different data types (text, vision, audio) into a single, cohesive model that has a deep, unified understanding of the world. 5. Practical Applications and Democratization The "how" of using AI is becoming as important as the "what." • Retrieval-Augmented Generation (RAG): This is a pivotal technique for making LLMs useful for businesses.

I

## 11. Interactive Chat


In [ ]:
# Interactive chat loop
def interactive_chat():
    print("\n🤖 RAG Chatbot Ready! Type 'quit' to exit.\n")

    while True:
        question = input("You: ")

        if question.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye! 👋")
            break

        if question.strip():
            try:
                result = rag_pipeline(question, embedding_model, index, chunks, model, tokenizer)
                print(f"\n🤖 Bot: {result['answer']}")
                print(f"📚 Sources: {', '.join(result['sources'])}\n")
            except Exception as e:
                print(f"❌ Error: {str(e)}\n")

# Uncomment the line below to start interactive chat
interactive_chat()



🤖 RAG Chatbot Ready! Type 'quit' to exit.

You: what is XAI?

🤖 Bot: Context: efficient "fine-tuned" models that excel at a particular job (e.g., legal document review, medical diagnosis assistance) at a fraction of the cost. This is often called "Small Language Models (SLMs)". • AI Agents: The next step is moving from a conversational chatbot to an autonomous agent. These are AI systems that can be given a high-level goal (e.g., "Plan a 5-day vacation to Tokyo"), and then independently break it down into steps: researching flights, booking hotels, creating an itinerary, and making restaurant reservations by using tools and APIs. 2. The Rise of "Responsible AI" and Governance As AI becomes more powerful, the backlash and concern about its ethical implications have grown exponentially. This is no longer an afterthought but a core business requirement. • Explainable AI (XAI): There's a massive push to make the "black box" of complex models more transparent. Why did the model make that d